In [1]:
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import copy
import itertools
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))

# Implicit Function Theorem

The implicit function theorem is a core ingredient in bifurcation theory, in this notebook we will explore how to apply it algorithmically 

## Statement of theorem

We will consider functions of the form:

$$f : \mathbb{R}^{n+m} \rightarrow \mathbb{R}^m $$

usually k-times continuously differentiable for $k \in \mathbb{N}$

The implicit function theorem is used to characterise the zero set of the function around a know solution, suppose (without loss of generality) that:

$$f(0) = 0$$

We then split $\mathbb{R}^n$ into the product of two spaces:

$$\mathbb{R}^{n+m} = \mathbb{R}^n \times \mathbb{R}^m$$ 

From here on we denote $\mathbb{R}^n = X$ and $\mathbb{R}^m = Y$

So that:

$$f : X \times Y \rightarrow \mathbb{R}^m$$

If we then suppose that the derivtative with respect to $Y$ is an isomorphism at 0, i.e:

$$\partial_{Y}f(0,0)$$ is invertible

Then the Implict Function Theorem states that there exists, open subsets of $X$ and $Y$ and a k-times differentiable function:

$$h : U_{X} \rightarrow U_{y}$$

That paramatrises the zero set of $f$ close to zero:

$$\{ (x,y) \in U_{X} \times U_{Y} \;| \;\; f(x,y) = 0 \} = \{ (x,h(x)) \;\; | \;\; x \in U_{X} \}$$

## Computing $h$

We will now construct an algorithm, that can produce a Taylor polynomial approximation of the $h$ function, assuming we know that $\mathbb{R}^{n+m}$ meets the conditions required.

Recall that:

$$h(x) = h(a) + \frac{h'(a)}{1!}(x-a) + \frac{h(a)}{2!}(x-a)^2 + \frac{h'(a)}{3!}(x-a)^3+\dotsb = \sum_{k=0}^\infty \frac{h^{\left(k\right)}(a)}{k!} (x-a)^k$$

is the k-th order Taylor expansion of $h$ at 0

We require a way to find the derivatives of $h$, specifically:

$\partial_{X}h$,     $\partial_{XX}h$,     $\partial_{XXX}h$ 

and so on

To do this observe that since $f(x, h(x)) = 0 \;\;\forall x \in U_{X}$

$$ \partial_{X}^{k}f(x,h(x)) = 0 \;\; \forall k \in \mathbb{N}, \forall x \in U_{X}$$

The first couple of applications of this idea are:

$$ 0 = f(x, h(x))$$

$$ 0 = f_{X} + f_{Y}h_{X}$$ 

$$ 0 = f_{XX} + 2f_{XY}h_{X} + f_{YY}h_{X}^2 + f_{Y}h_{XX}$$

(now omitting inner variables for brevity) 

https://math.stackexchange.com/questions/2037753/implicit-function-theorem-second-derivative-calculation-help

By assumption $f_{Y}$ is always invertible, so we can solve for the values of $h_{X}$ and $h_{XX}$.

Note that:

$$ h(0) = 0$$ 

since it can't be anything else, otherwise $(0,0)$, wouldn't be in the zero set - a contradiction

### Iterating the derivative

First we consider how to generate equation of the form above, we can split each term at each plus sign into a generalised term of the form:


$$ c \; f_{S_{1}S_{2} \cdots S_{k}}(x,h(x)) \cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right] $$

Where:

$$ S_{i} \in \{X,Y\} $$

And the $P_{i}$'s are either the identity map: $id: X \rightarrow X$, or a $l_{i}$th derivative of $h$

The $X_{\cdots}$ maps indicate where to put the input spaces' values, the $P_{i}$ maps can be viewed as "premultiples", preparing the input from: $X \times \cdots \times X$ to be fed in the $f$ partial.

Previously little attention was payed to the ordering, but now it is imperative to keep track of how the derivatives develop.

Translating the earlier work into this format (now omit $(x,h(x))$ for brevity):

$$ 0 = f$$

$$ 0 = f_{X} \cdot \left[ id(X_{1}) \right] + f_{Y} \cdot \left[ (h_{X}(X_{1}) \right]$$ 

$$ 0 = f_{XX} \cdot \left[ id(X_{1}), id(X_{2}) \right] + f_{XY} \cdot \left[ id(X_{1}),h_{X}(X_{2}) \right] + f_{YX}\left[ h_{X}(X_{1}),id(X_{2}) \right]+ f_{YY} \left[ h_{X}(X_{1}), h_{X}(X_{2}) \right] + f_{YY} \cdot \left[ (h_{XX}(X_{1}, X_{2}) \right] $$

Note: here taking a further derivative is indicated by appending $X$ or $Y$ to the end, contrary to common use, the reason for this is that it makes the book-keeping a lot easier

If we can form an encoding scheme for terms like this, and a way to differentiate them with respect to $X$ then we have a method to generate arbitrarlily high order equations to solve.

#### Differentiating

We consider:

$$ \partial_{X} \left( c \; f_{S_{1}S_{2} \cdots S_{k}} \cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right] \right)$$


$$ = c \left[ \left( \partial_{X}f_{S_{1}S_{2} \cdots S_{k}}  \right)\cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right] + f_{S_{1}S_{2} \cdots S_{k}} \cdot \left( \partial_{X} \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right]  \right) \right] $$

Splitting this into the two expressions either side of the $+$

$$  \left( \partial_{X}f_{S_{1}S_{2} \cdots S_{k}}  \right)\cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right]  = f_{S_{1}S_{2} \cdots S_{k}X} \cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}), id(X_{n+1}) \right] + f_{S_{1}S_{2} \cdots S_{k}Y} \cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{n,1}}, \cdots, X_{i_{k,l_{k}}}), h_{X}(X_{n+1}) \right]
$$

This happens since the partial derivative is evaluated at $(x,h(x))$, $n$ was the order of the orginal function, i.e the maximum value for $X_{\cdots}$

And:

$$ f_{S_{1}S_{2} \cdots S_{k}} \cdot \left( \partial_{X} \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right]  \right) =  f_{S_{1}S_{2} \cdots S_{k}} \sum_{i = 1}^{j} { \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, (\partial_{X}P_{i})(X_{i_{i,1}}, \cdots, X_{i_{i,l_{i}}}, X_{n+1}) , \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right] } $$

Here we abuse notation by using $i$ twice, unfortunately we have run out the usual indices.

Observe that the $X_{k+1}$ is now embedded inside the pre-multiples.

When $P_{i}$ is the identity $\partial_{X}P_{i} = \partial_{X}id = 0$, and so as expected the sum only picks up non-trivial premultiples.


While very complicated, we see that all the terms we computed stay as sums of expressions of the form:

$$ c \; f_{S_{1}S_{2} \cdots S_{k}}(x,h(x)) \cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right] $$

So inductively we see how the code can proceed

#### Coding the expression block storage + representation

Now we build the framework to do this, to stay in the spirit of sage, a latex output will be avaible. For expressions of the form:

$$ c \; f_{S_{1}S_{2} \cdots S_{k}}(x,h(x)) \cdot \left[P_{1}(X_{i_{1,1}}, \cdots, X_{i_{1,l_{1}}}), \cdots, P_{k}(X_{i_{k,1}}, \cdots, X_{i_{k,l_{k}}}) \right] $$

Also implemented is the differentiation rule derived earlier, returning a list of ExpressionBlock objects

In [2]:
class ExpressionBlock:
    def __init__(self, c = 1, s_string = '', n = 0, p_dict = {}, xs_dict = {}, func1 = 'f', func2 = 'h', X = 'X', Y = 'Y'):
        self.c = c # constant
        self.s_string = s_string # partials, i.e. 'XXY'
        self.n = n # number of X inputs the expression has
        self.k = len(s_string) # even if no partials, still include an identity premultiple
        self.p_dict = p_dict # P functions, 0 --> id, otherwise is a partial of h
        self.xs_dict = xs_dict # which X's go into each P
        
        # the rest are just for the cosmetic output
        self.func1 = func1 
        self.func2 = func2
        self.X = X
        self.Y = Y
        
    def __str__(self):
        # overide representation so can be printed as latex
        if self.c == 0:
            return ""
        
        out = ""
        # add the constant
        if self.c != 1:
            out += str(self.c) + " "
            
        # add f
        out += self.func1
        
        # add partials if there are any
        if len(self.s_string) != 0:
            out += "_{"
            out += self.s_string
            out += "} "
            out += r"\cdot \left["
            

        for i in range(1,self.k + 1):
            if self.p_dict[i] == 0:
                out += "id("
            else:
                out += "h_{"
                out += "X"*len(self.xs_dict[i])
                out += "}("
                
            for xi in self.xs_dict[i]:
                out += "X_{" + str(xi) + "}, "
                
            # inner excess comma deletion
            out = out[:-2]
            
            out += "), "
        
        if len(self.s_string) != 0:
            out = out[:-2] # outer excess comma deletion
            out += r"\right]"       
            
                
        # and we are done, here just list out the h terms               
        return out

    def _latex_(self):
        # so works with lprint
        return str(self)
    
    def diff(self):
        # apply the rule we worked out before
        # returns a list of ExpressionBlock instances
        
        out = []
        # first half
        temp = copy.deepcopy(self)
        temp.n += 1
        temp.s_string += 'X'
        temp.k += 1
        temp.p_dict[temp.k] = 0 # identity
        temp.xs_dict[temp.k] = (temp.n,)
        out.append(temp)
        
        temp = copy.deepcopy(self)
        temp.n += 1
        temp.s_string += 'Y'
        temp.k += 1
        temp.p_dict[temp.k] = 1 # identity
        temp.xs_dict[temp.k] = (temp.n,)
        out.append(temp)
        
        # second half
        # loop over each term in the sum
        for i in range(1,self.k + 1):
            # ktuple = (gamma, k)
            # indexing starts from 0
            if self.p_dict[i] == 0:
                # differentiating id --> zeros out
                continue 
            
            temp = copy.deepcopy(self)
            temp.n += 1
            temp.p_dict[i] = temp.p_dict[i] + 1 # take a further derivative of h
            new_xs = list(temp.xs_dict[i])
            new_xs.append(temp.n)
            
            temp.xs_dict[i] = tuple(new_xs)

            out.append(temp)
            
        # and we are done
        return out
            
            
            

        
        
k1 = ExpressionBlock(c = 3, s_string = 'XYX', n = 5,
                     p_dict = {1 : 0, 2: 3, 3 : 0},
                     xs_dict = {1 : (1,), 2 : (3,4,5), 3 : (2,)},)
lprint(k1)
k2 = ExpressionBlock()
lprint(k2)
lprint(k2.diff())

#[lprint(i) for i in k2.diff()]

$ 3 f_{XYX} \cdot \left[id(X_{1}), h_{XXX}(X_{3}, X_{4}, X_{5}), id(X_{2})\right] $

$ f $

$ \left[f_{X} \cdot \left[id(X_{1})\right], f_{Y} \cdot \left[h_{X}(X_{1})\right]\right] $

We see that at least for $f$ the rule works, lets wrap this up into a new class that deals with multiple blocks

In [3]:
class Expression:
    def __init__(self, blocks = [ExpressionBlock()]):
        if blocks is None:
            self.blocks = [] # python reasons - list is mutable
        else:
            self.blocks = blocks
            
    def __str__(self):
        if not self.blocks:
            # empty list
            return ""
        out = ""
        for block in self.blocks:
            # should be an instance of the ExpressionBlock class
            out += str(block) + " + "
        
        out = out[:-3] # remove last plus
        
        return out
    
    def _latex_(self):
        # so works with lprint
        return str(self)
    
    
    def diff(self):
        # returns a new Expression object that is the partial x derivative of the old one
        out = []
        for block in self.blocks:
            block_list = block.diff()
            
            # append to the list
            out += block_list
            
        return Expression(blocks = out)
            
        
            
    

e = Expression(blocks = [k1])
lprint(e)
lprint(e.diff())

$ 3 f_{XYX} \cdot \left[id(X_{1}), h_{XXX}(X_{3}, X_{4}, X_{5}), id(X_{2})\right] $

$ 3 f_{XYXX} \cdot \left[id(X_{1}), h_{XXX}(X_{3}, X_{4}, X_{5}), id(X_{2}), id(X_{6})\right] + 3 f_{XYXY} \cdot \left[id(X_{1}), h_{XXX}(X_{3}, X_{4}, X_{5}), id(X_{2}), h_{X}(X_{6})\right] + 3 f_{XYX} \cdot \left[id(X_{1}), h_{XXXX}(X_{3}, X_{4}, X_{5}, X_{6}), id(X_{2})\right] $

This all seems to be working fine

In [4]:
b = ExpressionBlock()
lprint(b)

$ f $

Now put into an expresssion so we can differentiate:

In [5]:
e = Expression(blocks = [b])
lprint(e)

$ f $

In [6]:
lprint(e.diff())

$ f_{X} \cdot \left[id(X_{1})\right] + f_{Y} \cdot \left[h_{X}(X_{1})\right] $

In [7]:
lprint(e.diff().diff())

$ f_{XX} \cdot \left[id(X_{1}), id(X_{2})\right] + f_{XY} \cdot \left[id(X_{1}), h_{X}(X_{2})\right] + f_{YX} \cdot \left[h_{X}(X_{1}), id(X_{2})\right] + f_{YY} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2})\right] + f_{Y} \cdot \left[h_{XX}(X_{1}, X_{2})\right] $

We see that this is working as we expect, observe how the symmetry of the total expression is conserved

In [8]:
lprint(e.diff().diff().diff())

$ f_{XXX} \cdot \left[id(X_{1}), id(X_{2}), id(X_{3})\right] + f_{XXY} \cdot \left[id(X_{1}), id(X_{2}), h_{X}(X_{3})\right] + f_{XYX} \cdot \left[id(X_{1}), h_{X}(X_{2}), id(X_{3})\right] + f_{XYY} \cdot \left[id(X_{1}), h_{X}(X_{2}), h_{X}(X_{3})\right] + f_{XY} \cdot \left[id(X_{1}), h_{XX}(X_{2}, X_{3})\right] + f_{YXX} \cdot \left[h_{X}(X_{1}), id(X_{2}), id(X_{3})\right] + f_{YXY} \cdot \left[h_{X}(X_{1}), id(X_{2}), h_{X}(X_{3})\right] + f_{YX} \cdot \left[h_{XX}(X_{1}, X_{3}), id(X_{2})\right] + f_{YYX} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2}), id(X_{3})\right] + f_{YYY} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2}), h_{X}(X_{3})\right] + f_{YY} \cdot \left[h_{XX}(X_{1}, X_{3}), h_{X}(X_{2})\right] + f_{YY} \cdot \left[h_{X}(X_{1}), h_{XX}(X_{2}, X_{3})\right] + f_{YX} \cdot \left[h_{XX}(X_{1}, X_{2}), id(X_{3})\right] + f_{YY} \cdot \left[h_{XX}(X_{1}, X_{2}), h_{X}(X_{3})\right] + f_{Y} \cdot \left[h_{XXX}(X_{1}, X_{2}, X_{3})\right] $

In [9]:
lprint(e.diff().diff().diff().diff())

$ f_{XXXX} \cdot \left[id(X_{1}), id(X_{2}), id(X_{3}), id(X_{4})\right] + f_{XXXY} \cdot \left[id(X_{1}), id(X_{2}), id(X_{3}), h_{X}(X_{4})\right] + f_{XXYX} \cdot \left[id(X_{1}), id(X_{2}), h_{X}(X_{3}), id(X_{4})\right] + f_{XXYY} \cdot \left[id(X_{1}), id(X_{2}), h_{X}(X_{3}), h_{X}(X_{4})\right] + f_{XXY} \cdot \left[id(X_{1}), id(X_{2}), h_{XX}(X_{3}, X_{4})\right] + f_{XYXX} \cdot \left[id(X_{1}), h_{X}(X_{2}), id(X_{3}), id(X_{4})\right] + f_{XYXY} \cdot \left[id(X_{1}), h_{X}(X_{2}), id(X_{3}), h_{X}(X_{4})\right] + f_{XYX} \cdot \left[id(X_{1}), h_{XX}(X_{2}, X_{4}), id(X_{3})\right] + f_{XYYX} \cdot \left[id(X_{1}), h_{X}(X_{2}), h_{X}(X_{3}), id(X_{4})\right] + f_{XYYY} \cdot \left[id(X_{1}), h_{X}(X_{2}), h_{X}(X_{3}), h_{X}(X_{4})\right] + f_{XYY} \cdot \left[id(X_{1}), h_{XX}(X_{2}, X_{4}), h_{X}(X_{3})\right] + f_{XYY} \cdot \left[id(X_{1}), h_{X}(X_{2}), h_{XX}(X_{3}, X_{4})\right] + f_{XYX} \cdot \left[id(X_{1}), h_{XX}(X_{2}, X_{3}), id(X_{4})\right] + f_{XYY} \cdot \left[id(X_{1}), h_{XX}(X_{2}, X_{3}), h_{X}(X_{4})\right] + f_{XY} \cdot \left[id(X_{1}), h_{XXX}(X_{2}, X_{3}, X_{4})\right] + f_{YXXX} \cdot \left[h_{X}(X_{1}), id(X_{2}), id(X_{3}), id(X_{4})\right] + f_{YXXY} \cdot \left[h_{X}(X_{1}), id(X_{2}), id(X_{3}), h_{X}(X_{4})\right] + f_{YXX} \cdot \left[h_{XX}(X_{1}, X_{4}), id(X_{2}), id(X_{3})\right] + f_{YXYX} \cdot \left[h_{X}(X_{1}), id(X_{2}), h_{X}(X_{3}), id(X_{4})\right] + f_{YXYY} \cdot \left[h_{X}(X_{1}), id(X_{2}), h_{X}(X_{3}), h_{X}(X_{4})\right] + f_{YXY} \cdot \left[h_{XX}(X_{1}, X_{4}), id(X_{2}), h_{X}(X_{3})\right] + f_{YXY} \cdot \left[h_{X}(X_{1}), id(X_{2}), h_{XX}(X_{3}, X_{4})\right] + f_{YXX} \cdot \left[h_{XX}(X_{1}, X_{3}), id(X_{2}), id(X_{4})\right] + f_{YXY} \cdot \left[h_{XX}(X_{1}, X_{3}), id(X_{2}), h_{X}(X_{4})\right] + f_{YX} \cdot \left[h_{XXX}(X_{1}, X_{3}, X_{4}), id(X_{2})\right] + f_{YYXX} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2}), id(X_{3}), id(X_{4})\right] + f_{YYXY} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2}), id(X_{3}), h_{X}(X_{4})\right] + f_{YYX} \cdot \left[h_{XX}(X_{1}, X_{4}), h_{X}(X_{2}), id(X_{3})\right] + f_{YYX} \cdot \left[h_{X}(X_{1}), h_{XX}(X_{2}, X_{4}), id(X_{3})\right] + f_{YYYX} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2}), h_{X}(X_{3}), id(X_{4})\right] + f_{YYYY} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2}), h_{X}(X_{3}), h_{X}(X_{4})\right] + f_{YYY} \cdot \left[h_{XX}(X_{1}, X_{4}), h_{X}(X_{2}), h_{X}(X_{3})\right] + f_{YYY} \cdot \left[h_{X}(X_{1}), h_{XX}(X_{2}, X_{4}), h_{X}(X_{3})\right] + f_{YYY} \cdot \left[h_{X}(X_{1}), h_{X}(X_{2}), h_{XX}(X_{3}, X_{4})\right] + f_{YYX} \cdot \left[h_{XX}(X_{1}, X_{3}), h_{X}(X_{2}), id(X_{4})\right] + f_{YYY} \cdot \left[h_{XX}(X_{1}, X_{3}), h_{X}(X_{2}), h_{X}(X_{4})\right] + f_{YY} \cdot \left[h_{XXX}(X_{1}, X_{3}, X_{4}), h_{X}(X_{2})\right] + f_{YY} \cdot \left[h_{XX}(X_{1}, X_{3}), h_{XX}(X_{2}, X_{4})\right] + f_{YYX} \cdot \left[h_{X}(X_{1}), h_{XX}(X_{2}, X_{3}), id(X_{4})\right] + f_{YYY} \cdot \left[h_{X}(X_{1}), h_{XX}(X_{2}, X_{3}), h_{X}(X_{4})\right] + f_{YY} \cdot \left[h_{XX}(X_{1}, X_{4}), h_{XX}(X_{2}, X_{3})\right] + f_{YY} \cdot \left[h_{X}(X_{1}), h_{XXX}(X_{2}, X_{3}, X_{4})\right] + f_{YXX} \cdot \left[h_{XX}(X_{1}, X_{2}), id(X_{3}), id(X_{4})\right] + f_{YXY} \cdot \left[h_{XX}(X_{1}, X_{2}), id(X_{3}), h_{X}(X_{4})\right] + f_{YX} \cdot \left[h_{XXX}(X_{1}, X_{2}, X_{4}), id(X_{3})\right] + f_{YYX} \cdot \left[h_{XX}(X_{1}, X_{2}), h_{X}(X_{3}), id(X_{4})\right] + f_{YYY} \cdot \left[h_{XX}(X_{1}, X_{2}), h_{X}(X_{3}), h_{X}(X_{4})\right] + f_{YY} \cdot \left[h_{XXX}(X_{1}, X_{2}, X_{4}), h_{X}(X_{3})\right] + f_{YY} \cdot \left[h_{XX}(X_{1}, X_{2}), h_{XX}(X_{3}, X_{4})\right] + f_{YX} \cdot \left[h_{XXX}(X_{1}, X_{2}, X_{3}), id(X_{4})\right] + f_{YY} \cdot \left[h_{XXX}(X_{1}, X_{2}, X_{3}), h_{X}(X_{4})\right] + f_{Y} \cdot \left[h_{XXXX}(X_{1}, X_{2}, X_{3}, X_{4})\right] $

### How to evaluate?

This is a good exercise, but how can we use this to get the Taylor polynomial of f?, for each differentiation level, observe each term can be evaluated at a constant number of vectors $v \in X$

Recalling that we can view derivatives as linear maps

### Extracting the Taylor polynomial

We want to get a polynomial from the k-th partial of $h$, to do this we evaluate on variants of the basis vector to get coefficients of the term in the polynomial, for example; if we want to find the coefficient of:

$$x_{1}^2 x_{3}$$

then we would evaluate:


$$h_{XXX}(e_{1}, e_{1}, e_{3})$$

Well almost, in fact we would need to combine the coefficients of:

$$x_{1} x_{1} x_{3}, \;\; x_{1} x_{3} x_{1}, \;\; x_{3} x_{1} x_{1}$$

Summming them

See that the formula for the number of these evaluations is:

$$\frac{3!}{2! 1!}$$

Since we can view as a permuation of $S_n$ where some are indistinguishable since variable have the same name

In general:

$$n_{\text{evals}}\left(\prod_{i = 1}^{n} {x_{i}^{k_{i}}}\right) = \frac{n!}{k_{1}! \cdots k_{n}!} $$

But by symmetry of the derivative multilinear map we only need to evaluate once, then multiple by this $n_{\text{evals}}$ value and we are done

In [10]:
# TODO will need to use tensors
# since h will lead to arbitrary elements of Y
# numpy implementation probably the best way to go
# but will need a way to extract partial information in a sensible way to build the tensor

### Tensor implementation

I will implement a symbolic tensor class, interestingly the pandas dataframe class supports both multindexing and custom index collapsing operations, as such I will store the tensor information in one of these for now

We will aim to mimic the functionality of the numpy tensor class, here we build a 3-dimensional tensor

In [11]:
x = np.array([[[56, 183, 1],
               [65, 164, 0]],
              [[85, 176, 1],
               [44, 164, 0]]])

In [12]:
print(x)

[[[ 56 183   1]
  [ 65 164   0]]

 [[ 85 176   1]
  [ 44 164   0]]]


In [13]:
x[0][1][1]

164

In [14]:
a = []
a.append(list(range(0, 3)))
print(a)

[[0, 1, 2]]


In [15]:
class SymbolicXYTensor():
        def __init__(self, x_dim, y_dim, xy_order):
            self.x_dim = x_dim
            self.y_dim = y_dim
            self.xy_order = xy_order
            self.data = pd.DataFrame
            
            # the sets from which we draw the possible multi-indices
            x_dims = list(range(1, x_dim+1)) 
            y_dims = list(range(1, y_dim+1))
            x_dims = ['x' + str(dim) for dim in x_dims]
            y_dims = ['y' + str(dim) for dim in y_dims]

            iterables = [] 

            self.size = 1
            for space in self.xy_order:
                if (space == 'x') or (space == 'X'):
                    iterables.append(x_dims)
                    self.size = self.size*x_dim 
                elif (space == 'y') or (space == 'Y'):
                    iterables.append(y_dims)
                    self.size = self.size*y_dim
                else:
                    raise(Exception('invalid xy_order syntax'))

            multindex = pd.MultiIndex.from_product(iterables, names = list(range(1, len(xy_order) + 1)))

            self.data = pd.DataFrame(pd.Series(np.zeros(self.size), index = multindex), columns = ['data'] )
            
        def fill_from_function(self, function, var_dict, position):
            def row_func(row):
                partials = row.name # a tuple
                temp = function # will be differentiating
                #print(row.name)
                for partial in partials:
                    temp = temp.diff(var_dict[partial])
                    #lprint(temp)
                return temp(*position) # unpack tuple as coordinates
            
            def row_func_no_eval(row):
                partials = row.name # a tuple
                temp = function # will be differentiating
                #print(row.name)
                for partial in partials:
                    temp = temp.diff(var_dict[partial])
                    #lprint(temp)
                return temp # this is used when we know will be a constant
            
            if position is None:
                # bit hacky
                self.data['data'] = self.data.apply(row_func_no_eval, axis = 1)
            else:
                self.data['data'] = self.data.apply(row_func, axis = 1)
            
        def vec_mult(self, vec):
            if len(self.xy_order) == 1:
                # dual space case
                #print(self.data)
                return sum(self.data['data']*vec)
            
            out = SymbolicXYTensor(x_dim = self.x_dim, y_dim = self.y_dim, xy_order = self.xy_order[:-1])
            
            out.data = self.data.copy(deep = True)
            out.data['vec'] = list(vec)*int(out.size)
            out.data['data'] = out.data['data']*out.data['vec']
            out.data = out.data.drop(columns = 'vec')
            
            #print('preparing to gb')
            #print(out.data)
            
            out.data = out.data.groupby(level=[Integer(i) for i in range(Integer(1),Integer(len(self.xy_order)))]).sum()
            
            #print('after gb')
            #print(self.xy_order)
            #print(out.data)
            
            return out
            # for some reason only sage integers work here, who knows why
   


In [16]:
var('x1 x2 x3 y1 y2')
f1(x1, x2, x3, y1, y2) = x1*x2*x3*y1 + cos(y2)
f2(x1, x2, x3, y1, y2) = sin(x1 + x2 + y1) + (x3 - y2)^3
lprint(f1)
lprint(f2)

$ \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ x_{1} x_{2} x_{3} y_{1} + \cos\left(y_{2}\right) $

$ \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ {\left(x_{3} - y_{2}\right)}^{3} + \sin\left(x_{1} + x_{2} + y_{1}\right) $

Lets try to producce teh $f_{XYX}$ tensor for the $f2$ component

In [17]:
a = SymbolicXYTensor(x_dim = 3, y_dim = 2,xy_order = 'XYX')

a.fill_from_function(f2, {'x1' : x1, 'x2' : x2, 'x3' : x3, 'y1' : y1, 'y2' : y2}, (1,1,1,1,1))

vec1 = (1,2,1)
vec2 = (-1,1)
vec3 = (1,2,3)

a.data.head()

data
1  2  3          
x1 y1 x1  -cos(3)
      x2  -cos(3)
      x3        0
   y2 x1        0
      x2        0

In [18]:
a.vec_mult(vec1).data # tensor-vector mutltiplication

data
1  2            
x1 y1  -3*cos(3)
   y2          0
x2 y1  -3*cos(3)
   y2          0
x3 y1          0
   y2         -6

In [19]:
a.vec_mult(vec1).vec_mult(vec2).data

,data
1,
x1,3*cos(3)
x2,3*cos(3)
x3,-6


In [20]:
lprint(a.vec_mult(vec1).vec_mult(vec2).vec_mult(vec3)) # final evaluation like dual space operation

$ 9 \, \cos\left(3\right) - 18 $

It works! we evaluated the slots of $f_{XYX}$ right from left

This tensor just got us the value for the second slot of the output, a similar process with the f1 function would get that

### Validating

Lets compare the usual numpy tensor implementation

In [21]:
x = np.array([[[1, 2, 1],
               [0, 0, 1]],
              [[-1, -1, 2],
               [1, 0, 3]]])

In [22]:
print(x)
print(x.shape)
print(np.tensordot(x, np.array([1,1,1]), axes = 1))

[[[ 1  2  1]
  [ 0  0  1]]

 [[-1 -1  2]
  [ 1  0  3]]]
(2, 2, 3)
[[4 1]
 [0 4]]


In [23]:
b = SymbolicXYTensor(x_dim = 3, y_dim = 2, xy_order = 'YYX')
b.data['data'] = [1, 2, 1, 0, 0, 1, -1, -1 , 2,0, 0 ,0 ]
b.data

data
1  2  3      
y1 y1 x1    1
      x2    2
      x3    1
   y2 x1    0
      x2    0
      x3    1
y2 y1 x1   -1
      x2   -1
      x3    2
   y2 x1    0
      x2    0
      x3    0

In [24]:
b.vec_mult([1,1,1]).data

data
1  2      
y1 y1    4
   y2    1
y2 y1    0
   y2    0

We see that the tensor-vector product works exactly like numpy's but entirely symbolically

## Producing Taylor Polynomials

Now that we have the requisite components it remains to contruct a method for producing a taylor polynomial for

$$ h : X \rightarrow Y $$

This is an iterative three step method:

1: compute the coefficients for the terms that $f_{y} \cdot [h_{kX}(X_{1},...,X_{k})] $ contributes (these will be kth order polynomial terms)

2: apply the linear transformation $f_{Y}^{-1} : Y \rightarrow Y$ to this polynomial function to get the relevant coefficients for $h_{kX}$ and so get the polynomial

3: save $h_{kX}$ to use in the next iteration

In [25]:
class SymbolicXYVectorTensor():
    # class that wraps up n tensors so they can represent multivariable domain functions
    # and evaluate to produce vectors as the lowest dimensional output
        def __init__(self, x_dim, y_dim, xy_order ='', vec_length = 1):
            
            self.x_dim = x_dim
            self.y_dim = y_dim
            self.xy_order = xy_order
            self.vec_length = vec_length
            
            self.tensors = {}
            for i in range(0, vec_length):
                self.tensors[i] = SymbolicXYTensor(x_dim, y_dim, xy_order)
                
                
        def fill_from_functions(self, functions, var_dict, position):
            for i in range(0, self.vec_length):
                self.tensors[i].fill_from_function(functions[i], var_dict, position)
                
        def vec_mult(self, vec):
            #print(self.xy_order)
            if len(self.xy_order) == 1:
                out = [] # just in the output is vector case
                for i in range(0, self.vec_length):
                    out.append(self.tensors[i].vec_mult(vec))
                    
            else:
                # returning a lower dimensional tensor
                out = SymbolicXYVectorTensor(x_dim = self.x_dim, y_dim = self.y_dim,
                                             xy_order = self.xy_order[:-1], vec_length = self.vec_length)
                
                #print(self.tensors[0].data)
                #print(out.tensors[0].data)
                
                for i in range(0, self.vec_length):
                    out.tensors[i] = self.tensors[i].vec_mult(vec)
                    #print(out.tensors[i].data)
                    #print('vec multed')
                    
            return out
        
        def evaluate(self, vectors):
            # evaluate all the way to vector output
            if len(vectors) != len(self.xy_order):
                raise(Exception('wrong number of vector inputs provided!'))
                return
            
            current_tensors = self
            for vec in vectors[::-1]:
                current_tensors = current_tensors.vec_mult(vec)
                # collapse down the SymbolicXYVectorTensor objects
                
            if len(current_tensors) != self.vec_length:
                raise(Exception('something has gone wrong!'))
                
            return current_tensors # should just be a vector by this point
                
                
                

In [26]:
class TensorDict(dict):
    # upgraded dictionary class that fills in missing elements using the rules
        def __init__(self, funcs, position, var_dict, x_dim = 2, y_dim = 2, func1 = 'f', func2 = 'h', X = 'X', Y = 'Y'):
            super(TensorDict, self).__init__() # initialise empty dictionary
            
            self.funcs = funcs # list of functions that we will evaluate to get the tensors
            self.position = position # where eveything is evalated (bifurcation point?)
            self.var_dict = var_dict # the sage variables we'll be using
            self.x_dim = x_dim
            self.y_dim = y_dim
            self.func1 = func1
            self.func2 = func2
            self.X = X
            self.Y = Y
            
        def __getitem__(self, key):
            # lookup a tensor, if its not there then build it
            try:
                return dict.__getitem__(self, key)
            except: # keyerror
                print('generating ' + key)
                tensor = SymbolicXYVectorTensor(x_dim = self.x_dim , y_dim = self.y_dim, xy_order = key[1:], vec_length = self.y_dim)
                # ignore the f/h at the start
                # this is the case for both derivatives of f,h : _ --> Y
                
                if key[0] == self.func1:
                    # just getting values of f_....
                    tensor.fill_from_functions(self.funcs, self.var_dict, self.position)
                elif key[0] == self.func2:
                    # this is h_kX derivative, not so easy to compute
                    raise(Exception(key + ' not possible, need to add the tensor dict manually'))
                    return 
                
                dict.__setitem__(self, key, tensor) # now save so we can reuse
                
                #print(tensor.tensors[0].data)
                return tensor
                
            
            
        

In [27]:
a = 'akrjgagj'
a[0]

'a'

## Testing

In [28]:
var('x1 x2 x3 y1 y2')
f1(x1, x2, x3, y1, y2) = x1*x2*x3*y1 + cos(y2)
f2(x1, x2, x3, y1, y2) = sin(x1 + x2 + y1) + (x3 - y2)^3
var_dict = {'x1' : x1, 'x2' : x2, 'x3' : x3, 'y1' : y1, 'y2' : y2}
lprint(f1)
lprint(f2)



$ \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ x_{1} x_{2} x_{3} y_{1} + \cos\left(y_{2}\right) $

$ \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ {\left(x_{3} - y_{2}\right)}^{3} + \sin\left(x_{1} + x_{2} + y_{1}\right) $

In [29]:
a = SymbolicXYVectorTensor(x_dim = 3, y_dim = 2, xy_order = 'XYX', vec_length = 2)
a.fill_from_functions([f1, f2], var_dict, (1,1,1,1,1) )
a.tensors[1].data.head()

data
1  2  3          
x1 y1 x1  -cos(3)
      x2  -cos(3)
      x3        0
   y2 x1        0
      x2        0

In [30]:
a.evaluate([(1,0,0), (1,1), (1,0,-1)])

[-1, -cos(3)]

In [31]:
a.tensors[0].vec_mult( (1,0,-1)).data

data
1  2      
x1 y1   -1
   y2    0
x2 y1    0
   y2    0
x3 y1    1
   y2    0

In [32]:
a.tensors[0].vec_mult( (1,0,-1)).vec_mult( (1,1)).data

,data
1,
x1,-1
x2,0
x3,1


In [33]:
a.tensors[0].vec_mult( (1,0,-1)).vec_mult( (1,1)).vec_mult( (1, 0, 0) )

-1

It looks like the vector output tensors behave just as expected

In [34]:
e = ExpressionBlock()
e = e.diff()[0]
lprint(e)

$ f_{X} \cdot \left[id(X_{1})\right] $

In [35]:
t_dict = TensorDict(funcs = [f1, f2], position = (1,1,1,1,1), var_dict = var_dict, x_dim = 3, y_dim = 2)
t_dict

{}

In [36]:
t_dict['fXX'].tensors[1].data

generating fXX


data
1  2          
x1 x1  -sin(3)
   x2  -sin(3)
   x3        0
x2 x1  -sin(3)
   x2  -sin(3)
   x3        0
x3 x1        0
   x2        0
   x3        0

### function to invert f_y

In [37]:
def get_fy_inv(x_dim, y_dim, funcs, var_dict, position):
    # first use the symbolic tensor class to compute the derivative
    A = SymbolicXYVectorTensor(x_dim, y_dim, xy_order = 'Y', vec_length = y_dim)
    A.fill_from_functions(funcs, var_dict, position)
    
    # now extract and arrange in the format of a sage matrix
    # each element in the tensor list will correspond to a row in the matrix
    
    row_list = []
    for index in A.tensors:
        row_list.append(A.tensors[index].data['data'].to_list())
        
    return matrix(row_list).inverse()

In [38]:
funcs = [f1, f2]
lprint(funcs[0])
lprint(funcs[1])

$ \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ x_{1} x_{2} x_{3} y_{1} + \cos\left(y_{2}\right) $

$ \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ {\left(x_{3} - y_{2}\right)}^{3} + \sin\left(x_{1} + x_{2} + y_{1}\right) $

In [39]:
lprint(get_fy_inv(3, 2, funcs, var_dict, (1,1,1,1,1)))

$ \left(\begin{array}{rr}
0 & \frac{1}{\cos\left(3\right)} \\
-\frac{1}{\sin\left(1\right)} & \frac{1}{\cos\left(3\right) \sin\left(1\right)}
\end{array}\right) $

In [40]:
# need a block to vector polynomial function, then can sum those over the blocks

In [93]:
def evaluate(e_block, vec_list, tensor_dict):

    # first prepare the inputs
    premultiplied = [] # list of the premultiplied vectors
    for i in e_block.p_dict.keys():
        # loop over the premultiples
        if e_block.p_dict[i] == 0:
            # identity premultiple case
            premultiplied.append(vec_list[e_block.xs_dict[i][0] - 1]) # -1 for list indexing
        else:
            # now for the case when have to evalate at h_kx
            inputs = [] # these are the inputs for h_kx
            for j in e_block.xs_dict[i]:
                inputs.append(vec_list[j - 1])

            h_kx = tensor_dict[e_block.func2 + e_block.p_dict[i]*e_block.X] # use the strings to build key
            # h_kx is a SymbolicXYVectorTensor
            premultiplied.append(h_kx.evaluate(inputs))
    
            
    return tensor_dict[e_block.func1 + e_block.s_string].evaluate(premultiplied)
    

In [94]:
def block_to_polynomial(y_dim, e_block, x_var_keys, var_dict, tensor_dict):
    
    order = e_block.n # the order of the polynomial terms
    
    # prepare list of polynomials, right now just the zero polynomial
    out = [0]*y_dim
    
    # build the basis vectors
    ei_vecs = {}
    link_dict = {}
    for i in range(1, len(x_var_keys) + 1):
        ei_vecs['e' + str(i)] = [int(i == j) for j in range(1, len(x_var_keys) + 1)]
        link_dict['e' + str(i)] = 'x' + str(i)
    
    # compute each term and add to out
    for combo in itertools.combinations_with_replacement(list(ei_vecs.keys()), order):
        #print([ei_vecs[key] for key in combo])
        coeffs = evaluate(e_block, [ei_vecs[key] for key in combo], tensor_dict) # evaluate at this combo of basis vectors
        term = prod([var_dict[link_dict[ei_key]] for ei_key in combo]) # xi*xj*xk^2 etc
        
        out = [a + b*term for a, b in zip(out, coeffs)] # update the output list
        
    return out    
    
    

In [95]:
e = ExpressionBlock()
e = e.diff()[0]
x_var_keys = ['x1', 'x2', 'x3']
t_dict = TensorDict(funcs = [f1, f2], position = (sqrt(pi),2,-1,1,0), var_dict = var_dict, x_dim = 3, y_dim = 2)
# it is the initialisation of the tensor dict that specifies the vast majority of the things we care about

In [96]:
lprint(e)
print(var_dict)
print(t_dict)
print(x_var_keys)

$ f_{X} \cdot \left[id(X_{1})\right] $

{'x1': x1, 'x2': x2, 'x3': x3, 'y1': y1, 'y2': y2}
{}
['x1', 'x2', 'x3']


In [97]:
poly_block = block_to_polynomial(2, e, x_var_keys, var_dict, t_dict)
lprint(poly_block)

generating fX


$ \left[-\sqrt{\pi} x_{2} + 2 \, \sqrt{\pi} x_{3} - 2 \, x_{1}, x_{1} \cos\left(\sqrt{\pi} + 3\right) + x_{2} \cos\left(\sqrt{\pi} + 3\right) + 3 \, x_{3}\right] $

In [98]:
lprint(funcs)

$ \left[\left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ x_{1} x_{2} x_{3} y_{1} + \cos\left(y_{2}\right), \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ {\left(x_{3} - y_{2}\right)}^{3} + \sin\left(x_{1} + x_{2} + y_{1}\right)\right] $

We see that this works, able to evaluate the terms in closed form, here the list is viewed as the $y_{1}, y_{2}$ vector

### Applying f_y inverse to the vector polynomial output

In [65]:
A = (get_fy_inv(3, 2, funcs, var_dict, (sqrt(pi),2,-1,1,0)))
lprint(A)

$ \left(\begin{array}{rr}
-\frac{1}{2 \, \sqrt{\pi}} & 0 \\
-\frac{\cos\left(\sqrt{\pi} + 3\right)}{6 \, \sqrt{\pi}} & -\frac{1}{3}
\end{array}\right) $

In [66]:
poly = list(A*vector(poly_block))
lprint(poly)

$ \left[\frac{\sqrt{\pi} x_{2} - 2 \, \sqrt{\pi} x_{3} + 2 \, x_{1}}{2 \, \sqrt{\pi}}, -\frac{1}{3} \, x_{1} \cos\left(\sqrt{\pi} + 3\right) - \frac{1}{3} \, x_{2} \cos\left(\sqrt{\pi} + 3\right) + \frac{{\left(\sqrt{\pi} x_{2} - 2 \, \sqrt{\pi} x_{3} + 2 \, x_{1}\right)} \cos\left(\sqrt{\pi} + 3\right)}{6 \, \sqrt{\pi}} - x_{3}\right] $

Its messy but stays in closed form

In [67]:
# then can build h_kx tensor from the resultant polynomial since need for later uses

In [70]:
b = ExpressionBlock()
e = Expression(blocks = [b])
e = e.diff()
e = e.diff()
e = e.diff()
e = e.diff()
lprint(e.blocks[-1])

$ f_{Y} \cdot \left[h_{XXXX}(X_{1}, X_{2}, X_{3}, X_{4})\right] $

In [99]:
def get_hkx_polynomial(funcs, k, x_dim, y_dim, var_dict, x_var_keys, tensor_dict, position, output = True):
    # note doesn't support constant c yet - but this doesn't appear so we are good
    
    if position != tensor_dict.position:
        raise(Exception('tensor dict position does not match position given'))
        
    if k == 0:
        # don't need to bother here, assuming that h(0) = 0
        return
       
    # inductively compute the lower orders of hkx so they will be added to the tensor dict
    get_hkx_polynomial(funcs, k-1, x_dim, y_dim, var_dict, x_var_keys, tensor_dict, position, output = False)
    # output is false so just updates the tensor dict
    
    # now ready to solve for hkx
    # first get the expression we need
    
    b = ExpressionBlock() # defaults to just f
    e = Expression(blocks = [b]) # ready to differentiate
    for i in range(0,k):
        e = e.diff()
        
    # the last block is hkx
    out = [0]*y_dim
    for block in e.blocks[:-1]:
        #lprint(block)
        new_term = block_to_polynomial(y_dim, block, x_var_keys, var_dict, tensor_dict)
        out = [a + b for a, b in zip(out, new_term)] # update the output list
        
    # multiply by fyinv
    A = get_fy_inv(x_dim, y_dim, funcs, var_dict, position)
    out = list(A*vector(out))
    
    # now convert to a tensor
    tensor = SymbolicXYVectorTensor(x_dim = x_dim , y_dim = y_dim, xy_order = 'X'*k, vec_length = y_dim)
    tensor.fill_from_functions(out, var_dict, position = None) # so avoids messy function evaluation
    tensor_dict['h' + 'X'*k] = tensor
    
    if output == True:
        return out
    
    return

In [100]:
lprint(funcs)
print(x_var_keys)
t_dict = TensorDict(funcs = [f1, f2], position = (0,0,0,1,1), var_dict = var_dict, x_dim = 3, y_dim = 2)
print(var_dict)

$ \left[\left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ x_{1} x_{2} x_{3} y_{1} + \cos\left(y_{2}\right), \left( x_{1}, x_{2}, x_{3}, y_{1}, y_{2} \right) \ {\mapsto} \ {\left(x_{3} - y_{2}\right)}^{3} + \sin\left(x_{1} + x_{2} + y_{1}\right)\right] $

['x1', 'x2', 'x3']
{'x1': x1, 'x2': x2, 'x3': x3, 'y1': y1, 'y2': y2}


In [101]:
lprint(get_hkx_polynomial(funcs, 1, 3, 2, var_dict, x_var_keys, t_dict, (0,0,0,1,1)))

generating fX


$ \left[\frac{x_{1} \cos\left(1\right) + x_{2} \cos\left(1\right) + 3 \, x_{3}}{\cos\left(1\right)}, 0\right] $

In [102]:
t_dict

{'fX': <__main__.SymbolicXYVectorTensor object at 0x6fff23879650>, 'hX': <__main__.SymbolicXYVectorTensor object at 0x6fff237c01d0>}

In [103]:
t_dict['hX'].tensors[0].data

,data
1,
x1,1
x2,1
x3,3/cos(1)


In [104]:
lprint(get_hkx_polynomial(funcs, 2, 3, 2, var_dict, x_var_keys, t_dict, (0,0,0,1,1)))

generating fXX
generating fXY
generating fYX
generating fYY


$ \left[-\frac{4 \, x_{1}^{2} \sin\left(1\right) + 4 \, x_{1} x_{2} \sin\left(1\right) + 4 \, x_{2}^{2} \sin\left(1\right) + 6 \, x_{3}^{2} + \frac{6 \, x_{1} x_{3} \sin\left(1\right)}{\cos\left(1\right)} + \frac{6 \, x_{2} x_{3} \sin\left(1\right)}{\cos\left(1\right)} + \frac{9 \, x_{3}^{2} \sin\left(1\right)}{\cos\left(1\right)^{2}}}{\cos\left(1\right)}, 0\right] $

In [105]:
lprint(get_hkx_polynomial(funcs, 3, 3, 2, var_dict, x_var_keys, t_dict, (0,0,0,1,1)))

generating fXXX
generating fXXY
generating fXYX
generating fXYY
generating fYXX
generating fYXY
generating fYYX
generating fYYY


$ \left[-\frac{3 \, x_{1} x_{2} x_{3}}{\cos\left(1\right) \sin\left(1\right)} - \frac{8 \, x_{1}^{3} \cos\left(1\right) + 8 \, x_{1}^{2} x_{2} \cos\left(1\right) + 8 \, x_{1} x_{2}^{2} \cos\left(1\right) + 8 \, x_{2}^{3} \cos\left(1\right) - \frac{12 \, x_{1} x_{3}^{2} {\left(\frac{3 \, \sin\left(1\right)}{\cos\left(1\right)^{2}} + 2\right)} \sin\left(1\right)}{\cos\left(1\right)} - \frac{12 \, x_{2} x_{3}^{2} {\left(\frac{3 \, \sin\left(1\right)}{\cos\left(1\right)^{2}} + 2\right)} \sin\left(1\right)}{\cos\left(1\right)} - \frac{48 \, x_{1}^{3} \sin\left(1\right)^{2}}{\cos\left(1\right)} - \frac{32 \, x_{1}^{2} x_{2} \sin\left(1\right)^{2}}{\cos\left(1\right)} - \frac{32 \, x_{1} x_{2}^{2} \sin\left(1\right)^{2}}{\cos\left(1\right)} - \frac{48 \, x_{2}^{3} \sin\left(1\right)^{2}}{\cos\left(1\right)} + 12 \, x_{1}^{2} x_{3} + 12 \, x_{1} x_{2} x_{3} + 12 \, x_{2}^{2} x_{3} - 6 \, x_{3}^{3} - \frac{54 \, x_{3}^{3} {\left(\frac{3 \, \sin\left(1\right)}{\cos\left(1\right)^{2}} + 2\right)} \sin\left(1\right)}{\cos\left(1\right)^{2}} - \frac{48 \, x_{1}^{2} x_{3} \sin\left(1\right)^{2}}{\cos\left(1\right)^{2}} - \frac{36 \, x_{1} x_{2} x_{3} \sin\left(1\right)^{2}}{\cos\left(1\right)^{2}} - \frac{48 \, x_{2}^{2} x_{3} \sin\left(1\right)^{2}}{\cos\left(1\right)^{2}} + \frac{18 \, x_{1} x_{3}^{2}}{\cos\left(1\right)} + \frac{18 \, x_{2} x_{3}^{2}}{\cos\left(1\right)} - \frac{36 \, x_{1} x_{3}^{2} \sin\left(1\right)^{2}}{\cos\left(1\right)^{3}} - \frac{36 \, x_{2} x_{3}^{2} \sin\left(1\right)^{2}}{\cos\left(1\right)^{3}} + \frac{27 \, x_{3}^{3}}{\cos\left(1\right)^{2}}}{\cos\left(1\right)}, -\frac{x_{1} x_{2} x_{3}}{\sin\left(1\right)}\right] $